In [1]:

%run ../chap01/abalone.ipynb

In [8]:
#메인함수 정의
def pulsar_exec(epoch_count=10, mb_size=10, report=1):
    load_pulsar_dataset()
    init_model()
    train_and_test(epoch_count, mb_size, report)

In [2]:
#데이터 적재함수 정의
def load_pulsar_dataset():
    with open(r'C:\Users\dhkdw\ipython\DeepLearning\chap02\pulsar_stars.csv') as csvfile:
        csvreader = csv.reader(csvfile)
        next(csvreader, None)
        rows=[]
        for row in csvreader:
            rows.append(row)
            
    global data, input_cnt, output_cnt
    input_cnt, output_cnt = 8,1
    data = np.asarray(rows, dtype = 'float32')

In [3]:
#후처리 과정에 대한 순전파와 역전파 함수의 재정의
def forward_postproc(output, y):
    entropy = sigmoid_cross_entropy_with_logits(y, output)
    loss = np.mean(entropy)
    return loss, [y, output, entropy]

def backprop_postproc(G_loss, aux):
    y, output, entropy = aux
    
    g_loss_entropy = 1.0 / np.prod(entropy.shape)
    g_entropy_output = sigmoid_cross_entropy_with_logits_derv(y, output)
    
    G_entropy = g_loss_entropy*G_loss
    G_output = g_entropy_output*G_entropy
    
    return G_output

In [4]:
#정확도 계산 함수의 재정의
def eval_accuracy(output, y):
    estimate = np.greater(output, 0)
    answer = np.greater(y, 0.5)
    correct = np.equal(estimate, answer)
    
    return np.mean(correct)

In [5]:
#시그모이드 관련 함수 정의
def relu(x):
    return np.maximum(x,0)
def sigmoid(x):
    return np.exp(-relu(-x))/(1.0+np.exp(-np.abs(x)))

def sigmoid_derv(x,y):
    return y*(1-y)

def sigmoid_cross_entropy_with_logits(z,x):
    return relu(x)- x * z + np.log(1 + np.exp(-np.abs(x)))

def sigmoid_cross_entropy_with_logits_derv(z, x):
    return -z + sigmoid(x)